<a href="https://colab.research.google.com/github/theoryxyz/llm/blob/main/ragchatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain langchain-community sentence-transformers faiss-cpu transformers accelerate bitsandbytes
!pip install -U bitsandbytes

In [ ]:
import torch
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from google.colab import drive # Google Driveをマウントするために必要

try:
    # Google Driveをマウント
    drive.mount('/content/drive', force_remount=True)
    print("Google Driveがマウントされました。")
except Exception as e:
    print(f"Google Driveのマウント中にエラーが発生しました: {e}")

print("\n準備ステップが完了しました。")

In [ ]:
# ドキュメントの読み込み
loader = TextLoader('/content/drive/My Drive/Colab Notebooks/ragchatbot/resume.txt')
documents = loader.load()

# ドキュメントを適切なサイズに分割
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
texts = text_splitter.split_documents(documents)

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
# 無償で利用できる、日本語に強い埋め込みモデルを指定
model_name = "intfloat/multilingual-e5-large"
embeddings = HuggingFaceEmbeddings(model_name=model_name)

# FAISSを使用してベクトルストアを作成
vectorstore = FAISS.from_documents(texts, embeddings)

In [ ]:
# Googleのオープンソースモデル "Gemma-2b-it" を指定
model_id = "google/gemma-2b-it"

# Hugging Faceトークンを明示的に渡す (notebook_login() 実行後に利用可能)
# from huggingface_hub import HfApi
# api = HfApi()
# token = api.token # または notebook_login() 後に自動で設定される環境変数などから取得

# トークナイザーとモデルを準備する際に、認証トークンを渡す
# もし notebook_login() で設定された環境変数等が自動で使われない場合、以下の token=... を有効にしてください
# 例: from huggingface_hub import HfFolder; token = HfFolder.get_token()

# モデルのトークナイザー（文章を単語に分割するツール）を準備
tokenizer = AutoTokenizer.from_pretrained(model_id) # , token=token を追加することも検討

# モデル本体を準備（4bit量子化でメモリ消費を抑える）
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    load_in_4bit=True
    # , token=token を追加することも検討
)

# LangChainで使えるように、Hugging Faceのpipeline機能でラップする
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512 # 最大出力トークン数
)

llm = HuggingFacePipeline(pipeline=pipe)

In [ ]:
# ベクトルストアを検索機（Retriever）として設定
retriever = vectorstore.as_retriever()

# RAGチェーンの作成
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True # 参考にした文章も表示する設定
)

# 質問を実行！
query = "望月さんの強みは何ですか？"
response = qa_chain.invoke(query)

print("【回答】")
print(response['result'])
print("\n【参考にした情報】")
for doc in response['source_documents']:
    print("- " + doc.page_content[:150] + "...")

In [ ]:
# 質問を実行！
query = "望月さんの開発エンジニアとしての強みは何ですか？"
response = qa_chain.invoke(query)

print("【回答】")
print(response['result'])
print("\n【参考にした情報】")
for doc in response['source_documents']:
    print("- " + doc.page_content[:150] + "...")